In [6]:
# First read in file
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

trendingSet = pd.read_csv('../data/new-datasets/trending_dataset.csv')
nontrendingSet = pd.read_csv('../data/new-datasets/nontrending_dataset.csv')
trendingDf = pd.DataFrame(trendingSet)
nontrendingDf = pd.DataFrame(nontrendingSet)

subsetTrending = trendingDf.head(6000)
subsetNontrending = nontrendingDf.head(len(subsetTrending))

#Concatenate data sets
fullSubset = pd.concat([subsetTrending, subsetNontrending])
fullSubset = fullSubset.drop_duplicates(subset='video_id', keep='first')
fullSubset['tags_length'] = np.array([int(i) if(i!='[0]') else 0 for i in fullSubset['tags_length'].iloc[:]])

#create labels
trendingLabels = np.array(fullSubset['trending'].iloc[:])
fullSubset = fullSubset.drop(columns=['tags','trending','duration','video_id','video_title'], axis=1)
matrixData = np.array(fullSubset)

#Standardize the data
matrixData = matrixData.astype('float32')
matrixData -= np.mean(matrixData, axis=0)
matrixData /= np.std(matrixData, axis=0)

# Start building the neural net
num_classes=2

model = keras.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(11900, 7)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(2, activation='softmax'))

#Train model
model.compile(optimizer=tf.train.AdamOptimizer(), loss='sparse_categorical_crossentropy', metrics=['accuracy']) #compile model
model.fit(matrixData, trendingLabels, epochs=50)

# trendingDf['tags_length'] = np.array([int(i) if(i!='[0]') else 0 for i in fullSubset['tags_length'].iloc[:]])
# nontrendingDf['tags_length'] = np.array([int(i) if(i!='[0]') else 0 for i in fullSubset['tags_length'].iloc[:]])
trendingTestSet = trendingDf.iloc[-400:].drop(columns=['tags','trending','duration','video_id','video_title'], axis=1)
nontrendingTestSet = nontrendingDf.iloc[-400:].drop(columns=['tags','trending','duration','video_id','video_title'], axis=1)
totaldf = pd.concat([trendingTestSet, nontrendingTestSet])
smallTestSet = np.array(totaldf)

#test set labels
trendingTestLabels = trendingDf.iloc[-400:]['trending']
nontrendingTestLabels = nontrendingDf.iloc[-400:]['trending']
# print(trendingTestLabels)
# print(nontrendingTestLabels)
totalTestlabels = np.concatenate([trendingTestLabels,nontrendingTestLabels])

#model test accuracy
_, testacc = model.evaluate(smallTestSet, totalTestlabels)
print("test accuracy is " + str(testacc))

#model predict 
# predictions = model.predict(smallTestSet)
# print(predictions)


ValueError: Input 0 of layer conv2d_4 is incompatible with the layer: expected ndim=4, found ndim=3. Full shape received: [None, 11900, 7]